https://medium.com/@tarekseif0/document-similarity-using-word-movers-distance-and-cosine-similarity-d698ad435422

In [1]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import json
from rake_nltk import Rake
from nltk.stem import WordNetLemmatizer 
from nltk import word_tokenize, pos_tag
from pyemd import emd
# from gensim.similarities import WmdSimilarity


# Load pretrained model (since intermediate data is not included, the model cannot be refined with additional data)
# model = Word2Vec.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True, norm_only=True)



In [2]:
model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True) #without *norm_only* param

In [3]:
# dog = model['dog']
# print(dog.shape)
# print(dog[:10])

# # Deal with an out of dictionary word: Михаил (Michail)
# # if 'Михаил' in model:
# #     print(model['Михаил'].shape)
# # else:
# #     print('{0} is an out of dictionary word'.format('Михаил'))

# # Some predefined functions that show content related information for given words
# print(model.most_similar(positive=['woman', 'king'], negative=['man']))

# # print(model.doesnt_match("breakfast cereal dinner lunch".split()))

# # print(model.similarity('woman', 'man'))

In [4]:
answers1 = []
f = open('answers1.txt')
lines = f.readlines()

for line in lines:
    if line!='\n':
        answers1 += [line.replace('\n', '')]

In [5]:
with open('model_answers.txt') as f: 
    data = f.read() 
  
print("Data type before reconstruction : ", type(data)) 
      
# reconstructing the data as a dictionary 
model_answer_list = json.loads(data) 
  
print("Data type after reconstruction : ", type(model_answer_list)) 
# print(model_answer_list) 

Data type before reconstruction :  <class 'str'>
Data type after reconstruction :  <class 'list'>


In [6]:
model_answer = model_answer_list[2]
# student_answer =  answers1[-2]
student_answer = "means first come first served .  It operating Systems scheduling algorithm that automatically executes queued requests and processes in order of their arrival Advantages is that it is simple and easy to understand  Disadvantages is that the process with less execution time suffer that waiting time is often quite lo long is"

print(model_answer)
print(student_answer)

{'FCFS stands for First Come, First Served.': 0.5, 'It is a type of scheduling algorithm.': 0.5, 'In this scheme, if a process requests the CPU first, it is allocated to the CPU first.': 1, 'Its implementation is managed by a FIFO queue.': 1, 'It is simple and easy to understand & implement.': 1, 'The process with less execution time suffer i.e. waiting time is often quite long.': 1, 'This effect results in lower CPU and device utilization.': 1, 'FCFS algorithm is particularly troublesome for time-sharing systems, where it is important that each user get a share of the CPU at regular intervals.': 1}
means first come first served .  It operating Systems scheduling algorithm that automatically executes queued requests and processes in order of their arrival Advantages is that it is simple and easy to understand  Disadvantages is that the process with less execution time suffer that waiting time is often quite lo long is


In [7]:
def penn_to_wn(tag):
    """ Convert between a Penn Treebank tag to a simplified Wordnet tag """
    if tag.startswith('N'):
        return 'n'
 
    if tag.startswith('V'):
        return 'v'
 
    if tag.startswith('J'):
        return 'a'
 
    if tag.startswith('R'):
        return 'r'
 
    return 'n'

In [8]:
def sentence_similarity(model_key, student_key):
    lemmatizer = WordNetLemmatizer() 

    model_answer_tokenized = word_tokenize(model_key)
    model_pos_tagged = pos_tag(model_answer_tokenized)
#     print(model_pos_tagged)
    model_tagged = [lemmatizer.lemmatize(word[0].lower(),penn_to_wn(word[1])) for word in model_pos_tagged]

    student_answer_tokenized = word_tokenize(student_key)
    student_pos_tagged = pos_tag(student_answer_tokenized)
#     print(student_pos_tagged)
    student_tagged = [lemmatizer.lemmatize(word[0].lower(),penn_to_wn(word[1])) for word in student_pos_tagged]
    
#     print(model_tagged,student_tagged)
    return model.wmdistance(model_tagged, student_tagged)

In [9]:
r = Rake()
model.init_sims(replace=True)

In [10]:
r.extract_keywords_from_text(student_answer)
student_keywords = r.get_ranked_phrases()
print(student_keywords)

['means first come first served', 'operating systems scheduling algorithm', 'often quite lo long', 'automatically executes queued requests', 'less execution time suffer', 'waiting time', 'understand disadvantages', 'arrival advantages', 'simple', 'processes', 'process', 'order', 'easy']


In [11]:
for sentence in model_answer.keys():
    sum = 0
    r.extract_keywords_from_text(sentence)
    sentence_keywords = r.get_ranked_phrases()
    print(sentence_keywords)
#     print(sentence_keyword,",",student_keyword,",",sentence_similarity(sentence_keyword, student_keyword))
    for sentence_keyword in sentence_keywords:
        best = None
        match = None
        for student_keyword in student_keywords:
#             print(student_keyword)
#             print(lemmatizer.lemmatize([x for x in word_tokenize(sentence_keyword)]))
#             print(sentence_keyword,",",student_keyword,",",sentence_similarity(sentence_keyword, student_keyword))
            sim = sentence_similarity(sentence_keyword, student_keyword)
            if best is None:
                best = sim
                match = student_keyword
            elif sim<best:
                best = sim
                match = student_keyword
        sum += best
        print(sentence_keyword,",",match,",", best)
    print(sum/len(sentence_keywords))

['first served', 'first come', 'fcfs stands']
first served , means first come first served , 0.5138518368579864
first come , means first come first served , 0.4880002018068313
fcfs stands , waiting time , 1.2559440785826443
0.7525987057491541
['scheduling algorithm', 'type']
scheduling algorithm , operating systems scheduling algorithm , 0.6121213810282647
type , simple , 1.2102627754211426
0.9111920782247036
['process requests', 'cpu first', 'scheme', 'allocated']
process requests , processes , 0.6488169431686401
cpu first , means first come first served , 0.8189030481029034
scheme , processes , 1.2096816301345825
allocated , automatically executes queued requests , 1.2844080323064029
0.9904524134281323
['fifo queue', 'managed', 'implementation']
fifo queue , waiting time , 1.1853999471887946
managed , easy , 1.2190260887145996
implementation , processes , 1.1563019752502441
1.1869093370512127
['understand', 'simple', 'implement', 'easy']
understand , understand disadvantages , 0.6540

In [13]:
model.save("word2vec.model")

In [13]:
# from gensim.similarities import WmdSimilarity
# lemmatizer = WordNetLemmatizer()

In [14]:
# num_best = 10
# instance = WmdSimilarity(student_keywords, model, num_best=10)

In [15]:
# for sentence in model_answer.keys():
#     r.extract_keywords_from_text(sentence)
#     sentence_keywords = r.get_ranked_phrases()
#     for sentence_keyword in sentence_keywords:
#         sentence_keyword_tokenized = word_tokenize(sentence_keyword)
#         sentence_keyword_pos_tagged = pos_tag(sentence_keyword_tokenized)
#         sentence_keyword_tagged = [lemmatizer.lemmatize(word[0].lower(),penn_to_wn(word[1])) for word in sentence_keyword_pos_tagged]
#         sims = instance[sentence_keyword_tagged]
#         print(sentence_keyword_tagged)
#         for i in range(len(sims)):
#             print ('sim = %.4f' % sims[i][1])
#             print(student_keywords[sims[i][0]])
# #         for student_keyword in student_keywords:

## Comparing average similarity of all answers with one-another

In [18]:
# from gensim.models import Word2Vec
# from gensim.models import KeyedVectors
# import json
# from rake_nltk import Rake
# from nltk.stem import WordNetLemmatizer 
# from nltk import word_tokenize, pos_tag
# from pyemd import emd
import math

In [17]:
# model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True) #without *norm_only* param

In [22]:
answers1 = []
f = open('answers1.txt',encoding='utf-8')
lines = f.readlines()

for line in lines:
    if line!='\n':
        answers1 += [line.replace('\n', '')]

In [23]:
with open('model_answers.txt') as f: 
    data = f.read() 
  
print("Data type before reconstruction : ", type(data)) 
      
# reconstructing the data as a dictionary 
model_answer_list = json.loads(data) 
  
print("Data type after reconstruction : ", type(model_answer_list)) 
# print(model_answer_list) 

Data type before reconstruction :  <class 'str'>
Data type after reconstruction :  <class 'list'>


In [24]:
def penn_to_wn(tag):
    """ Convert between a Penn Treebank tag to a simplified Wordnet tag """
    if tag.startswith('N'):
        return 'n'
 
    if tag.startswith('V'):
        return 'v'
 
    if tag.startswith('J'):
        return 'a'
 
    if tag.startswith('R'):
        return 'r'
 
    return 'n'

In [25]:
def sentence_similarity(model_key, student_key):
    lemmatizer = WordNetLemmatizer() 

    model_answer_tokenized = word_tokenize(model_key)
    model_pos_tagged = pos_tag(model_answer_tokenized)
    model_tagged = [lemmatizer.lemmatize(word[0].lower(),penn_to_wn(word[1])) for word in model_pos_tagged]

    student_answer_tokenized = word_tokenize(student_key)
    student_pos_tagged = pos_tag(student_answer_tokenized)
    student_tagged = [lemmatizer.lemmatize(word[0].lower(),penn_to_wn(word[1])) for word in student_pos_tagged]
    
#     print(model_tagged,student_tagged)
    return model.wmdistance(model_tagged, student_tagged)

In [24]:
r = Rake()
# model.init_sims(replace=True)
# model = Word2Vec.load("word2vec.model")
model = KeyedVectors.load("word2vec.model")

In [26]:
def evaluate(student_answer, model_answer):
    r.extract_keywords_from_text(student_answer)
    student_keywords = r.get_ranked_phrases()
#     print(student_keywords)
    marks = 0
    for sentence in model_answer.keys():
        sum = 0
        r.extract_keywords_from_text(sentence)
        sentence_keywords = r.get_ranked_phrases()
#         print(sentence_keywords)
        for sentence_keyword in sentence_keywords:
            best = None
            match = None
            for student_keyword in student_keywords:
                sim = sentence_similarity(sentence_keyword, student_keyword)
                if best is None:
                    best = sim
                    match = student_keyword
                elif sim<best:
                    best = sim
                    match = student_keyword
            if best != math.inf:
                sum += best
#                 print(sentence_keyword,",",match,",", best)
        if round(sum/len(sentence_keywords),1) <= 1:
            marks += model_answer[sentence]
        elif round(sum/len(sentence_keywords),1) > 1 and round(sum/len(sentence_keywords),1) <= 1.1:
            marks += model_answer[sentence] * 0.5
    print(marks)

In [27]:
for student_answer,model_answer in zip(answers1,model_answer_list):
    print("Student -\n",student_answer, "\nModel answer\n",''.join(model_answer.keys()))
    evaluate(student_answer, model_answer)

Student -
 Operating system main purpose is to manage all resources of hardware and software that are connect with computer. Without operating system all computer system are helpless, because operating system create the interface between user and hardware. 
Model answer
 It is designed to make sure that a computer system performs well by managing its computational activities.It provides an environment for the development and execution of programs.
1
Student -
 Mutual Exclusion is where a resource can be held by only one process at a time. Hold and Wait is where a process can hold a number of resources at a time and at the same time, it can request for other resources that are being held by some other process. No preemption is where a resource can't be preempted from the process by another process, forcefully.  Circular wait is a condition when the first process is waiting for the resource held by the second process, the second process is waiting for the resource held by the third proce

In [ ]:
from gensim.utils import lemmatize
lemmatized_sentence = [word.decode('utf-8').split('.')[0] for word in lemmatize("very economical idea")]
lemmatized_sentence

In [13]:
round(1.09,1)

1.1

In [22]:
!pip3 install --upgrade gensim --user